In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import glob
from tqdm import tqdm
from sklearn.metrics import confusion_matrix

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader as DataLoader

import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid

import image_show
from model_utils import *
from utils import *
from data_utils import create_validation_data

use_gpu = torch.cuda.is_available()
batch_size = 100
sz = 197
num_epoch = 10

In [6]:
use_gpu = torch.cuda.is_available()

In [7]:
Data_dir = "cifar_dataset"
train_dir = f'{Data_dir}/train' 
test_dir = f'{Data_dir}/test'

In [ ]:
train_data = glob.glob(f'{train_dir}/*/*.png')
valid_data = glob.glob(f'{test_dir}/*/*.png')

In [8]:
trfm = transforms.Compose([
    transforms.Resize((sz,sz)),   # PIL image object
    transforms.ToTensor(),        # Tensor
    transforms.Normalize([0.4913997551666284, 0.48215855929893703, 0.446530913373161],[0.24703225141799082, 0.24348516474564, 0.26158783926049628])
    ])

In [23]:
train_ds = datasets.ImageFolder(train_dir , transform = trfm)
test_ds = datasets.ImageFolder(test_dir , transform = trfm)

In [25]:
train_dl = DataLoader(train_ds , batch_size = batch_size ,
                      shuffle = True , num_workers = 8)
test_dl = DataLoader(test_ds , batch_size = batch_size ,
                      shuffle = True , num_workers = 8)

In [11]:
pretrained_model = models.resnet50(pretrained=False)

In [12]:
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, 10)

In [14]:
if use_gpu is True :
  pretrained_model = pretrained_model.cuda()

In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pretrained_model.parameters() , lr = 0.002 , momentum = 0.9)
schedular = optim.lr_scheduler.StepLR(optimizer , step_size = 1 , gamma = 0.95)

In [ ]:
losses = []

pretrained_model.train(True)

for epoch in range(num_epoch):
  schedular.step()

  for i,(input_img , labels) in enumerate(train_dl):
    
    input_img = to_var(input_img)
    labels = to_var(labels)
        
    # forward pass    computational graph for forward path is generated in this step and is an attribute of object "output"
    optimizer.zero_grad()
    output = pretrained_model(input_img)
        
    # loss
    loss = criterion(output , labels)  #computational graph will be completed in this step and will be transfered to next part as an attribute of object "loss" 
    losses.append(loss.data)
        
    # backward pass   This step will do backpropagation and result gradients
    loss.backward()
        
    # update weights
    optimizer.step()
        
    #report 
    print('epoch [%2d/%2d]  , step [%3d/%3d] , loss: %.4f'
          %(epoch + 1 , num_epoch , i+1 , len(train_ds)//batch_size , loss.data))

In [ ]:
plt.figure(figsize=(12, 7))
plt.plot(losses)
plt.xlabel('Iteration')
plt.ylabel('loss')
plt.title('Cross Entropy Loss')
plt.show()

In [20]:
def evaluate_model(model, dataloader):
    model.eval()  # for batch normalization layers
    corrects = 0
    with torch.no_grad():
      for inputs, targets in dataloader:
        #inputs, targets = to_var(inputs, True), to_var(targets, True)
        if use_gpu:
          inputs = inputs.cuda()
          targets = targets.cuda()

        outputs = model(inputs)
        _, preds = torch.max(outputs.data, 1)
        corrects += (preds == targets.data).sum()
    
    print('accuracy: {:.2f}'.format(100. * corrects / len(dataloader.dataset)))

In [26]:
evaluate_model(pretrained_model, test_dl)

accuracy: 95.82
